In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline

# Return Forecasting: Read Historical Daily Yen Futures Data
In this notebook, you will load historical Dollar-Yen exchange rate futures data and apply time series analysis and modeling to determine whether there is any predictable behavior.

In [ ]:
# Futures contract on the Yen-dollar exchange rate:
# This is the continuous chain of the futures contracts that are 1 month to expiration
yen_futures = pd.read_csv(
    Path("yen.csv"), index_col="Date", infer_datetime_format=True, parse_dates=True
)
yen_futures.head()

In [ ]:
# Trim the dataset to begin on January 1st, 1990
yen_futures = yen_futures.loc["1990-01-01":, :]
yen_futures.head()

 # Return Forecasting: Initial Time-Series Plotting

 Start by plotting the "Settle" price. Do you see any patterns, long-term and/or short?

In [ ]:
# Plot just the "Settle" column from the dataframe:
yen_futures.plot(y='Settle', title='Yen Futures Settle Price', figsize=(12,8))

---

# Decomposition Using a Hodrick-Prescott Filter

 Using a Hodrick-Prescott Filter, decompose the Settle price into a trend and noise.

In [ ]:
import statsmodels.api as sm

# Apply the Hodrick-Prescott Filter by decomposing the "Settle" price into two separate series:
ts_noise, ts_trend = sm.tsa.filters.hpfilter(yen_futures['Settle'])

In [ ]:
# Create a dataframe of just the settlereset_index add columns for "noise" and "trend" series from above:
df_yen_futures_Settle_stat = pd.concat([yen_futures['Settle'],ts_noise, ts_trend],axis=1)
df_yen_futures_Settle_stat = df_yen_futures_Settle_stat.rename(columns={'Settle_cycle':'noise','Settle_trend':'trend'})
df_yen_futures_Settle_stat.head()

In [ ]:
# Plot the Settle Price vs. the Trend for 2015 to the present
df_yen_Settle_priceVStrend_2015 = df_yen_futures_Settle_stat.loc['2015': ]
df_yen_Settle_priceVStrend_2015.plot(y=['Settle','trend'],title='Settle vs. Trend', figsize=(12,8))

In [ ]:
# Plot the Settle Noise
df_yen_futures_Settle_stat.plot(y='noise',title='Noise', figsize=(12,8))

---

# Forecasting Returns using an ARMA Model

Using futures Settle *Returns*, estimate an ARMA model

1. ARMA: Create an ARMA model and fit it to the returns data. Note: Set the AR and MA ("p" and "q") parameters to p=2 and q=1: order=(2, 1).
2. Output the ARMA summary table and take note of the p-values of the lags. Based on the p-values, is the model a good fit (p < 0.05)?
3. Plot the 5-day forecast of the forecasted returns (the results forecast from ARMA model)

In [ ]:
# Create a series using "Settle" price percentage returns, drop any nan"s, and check the results:
# (Make sure to multiply the pct_change() results by 100)
# In this case, you may have to replace inf, -inf values with np.nan"s
returns = (yen_futures[["Settle"]].pct_change() * 100)
returns = returns.replace(-np.inf, np.nan).dropna()
returns.tail()

In [ ]:
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)
# Estimate and ARMA model using statsmodels (use order=(2, 1))
from statsmodels.tsa.arima_model import ARMA
ARMA_model = ARMA(returns.values, order=(2,1))


# Fit the model and assign it to a variable called results
ARMA_results = ARMA_model.fit()


In [ ]:
# Output model summary results:
ARMA_summary = ARMA_results.summary()
ARMA_summary

In [ ]:
# Plot the 5 Day Returns Forecast
ARMA_plot = pd.DataFrame(ARMA_results.forecast(steps=5)[0]).plot(title="5  Day Returns Forecast")
ARMA_plot

---

# Forecasting the Settle Price using an ARIMA Model

 1. Using the *raw* Yen **Settle Price**, estimate an ARIMA model.
     1. Set P=5, D=1, and Q=1 in the model (e.g., ARIMA(df, order=(5,1,1))
     2. P= # of Auto-Regressive Lags, D= # of Differences (this is usually =1), Q= # of Moving Average Lags
 2. Output the ARIMA summary table and take note of the p-values of the lags. Based on the p-values, is the model a good fit (p < 0.05)?
 3. Construct a 5 day forecast for the Settle Price. What does the model forecast will happen to the Japanese Yen in the near term?

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

# Estimate and ARIMA Model:
# Hint: ARIMA(df, order=(p, d, q))
ARIMA_model = ARIMA(yen_futures['Settle'], order=(5, 1, 1))

# Fit the model
ARIMA_results = ARIMA_model.fit()

In [ ]:
# Output model summary results:
ARIMA_summary = ARIMA_results.summary()
ARIMA_summary

In [ ]:
# Plot the 5 Day Price Forecast
# Plot the 5 Day Price Forecast
ARIMA_plot = pd.DataFrame(ARIMA_results.forecast(steps=5)[0]).plot(title="5 Day Returns Price Forecast")
ARIMA_plot

---

# Volatility Forecasting with GARCH

Rather than predicting returns, let's forecast near-term **volatility** of Japanese Yen futures returns. Being able to accurately predict volatility will be extremely useful if we want to trade in derivatives or quantify our maximum loss.
 
Using futures Settle *Returns*, estimate an GARCH model

1. GARCH: Create an GARCH model and fit it to the returns data. Note: Set the parameters to p=2 and q=1: order=(2, 1).
2. Output the GARCH summary table and take note of the p-values of the lags. Based on the p-values, is the model a good fit (p < 0.05)?
3. Plot the 5-day forecast of the volatility.

In [ ]:
from arch import arch_model

In [ ]:
# Estimate a GARCH model:
model = arch_model(returns, mean="Zero", vol="GARCH", p=2, q=1)

# Fit the model
res = model.fit(disp="off")

In [ ]:
# Summarize the model results
res.summary()

In [ ]:
# Find the last day of the dataset
last_day = returns.index.max().strftime('%Y-%m-%d')
last_day

In [ ]:
# Create a 5 day forecast of volatility
forecast_horizon = 5
# Start the forecast using the last_day calculated above
forecasts = res.forecast(start=last_day, horizon=forecast_horizon)
forecasts

In [ ]:
# Annualize the forecast
intermediate = np.sqrt(forecasts.variance.dropna() * 252)
intermediate.head()

In [ ]:
# Transpose the forecast so that it is easier to plot
final = intermediate.dropna().T
final.head()

In [ ]:
# Plot the final forecast
final.plot(title='5 Day Forecast of Volatility')

---

# Conclusions

Based on your time series analysis, would you buy the yen now? Yes

Is the risk of the yen expected to increase or decrease?  increase

Based on the model evaluation, would you feel confident in using these models for trading? Yes